# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [ ]:
#Define the models to use
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

#Load the API key from the environment variable
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

#Check if the API key is valid
if api_key and api_key.startswith('sk-') and len(api_key) > 10:
    print("API key found")
else:
    print("No API key found or is not valid")

# Create an instance of the OpenAI client
openai_client = OpenAI(api_key=api_key)
print("OpenAI client instance created")

# Create an instance of the Ollama client
# ollama_client = OpenAI(base_url = OLLAMA_BASE_URL, api_key='ollama')
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
print("Ollama client instance created")


In [ ]:
# requests.get("http://localhost:11434").content

In [ ]:
# !ollama pull llama3.2

In [3]:
# set up environment
question_system_prompt = """
You are a helpful assistant that can explain technical concepts in a way that is easy to understand even for non-technical people.
With the help of the user, you will be able to explain the concept in a way that is easy to understand.
"""
question_user_prompt = """
Please explain what this code does and why:

def fetch_website_contents(url):
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]
# """


# question_user_prompt = """
# Please explain what the solution to this math question and also the reasoning behind it:

# You toss 2 coins. One of them is heads. What's the probability the other is tails? Answer with the probability only
# """

In [ ]:
# Get gpt-4o-mini to answer, with streaming
stream = openai_client.chat.completions.create(
    model = MODEL_GPT,
    messages = [
        {"role": "system", "content": question_system_prompt},
        {"role": "user", "content": question_user_prompt}
    ],
    stream=True
)

response = " "
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get Llama 3.2 to answer (use same client as in constants: ollama)
response = ollama.chat.completions.create(
    model = MODEL_LLAMA,
    messages = [
        {"role": "system", "content": question_system_prompt},
        {"role": "user", "content": question_user_prompt}
    ]
)
result = response.choices[0].message.content
display(Markdown(result))